## Superstore Sales Data Cleaning.

#### Importing python libraries for data analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Importing superstore sales data into python.

In [ ]:
orders = pd.read_excel('Superstore Sales Orders Data.xlsx')
people = pd.read_excel('Superstore Sales People Data.xlsx')
returns = pd.read_excel('Superstore Sales Returns Data.xlsx')

#### Reading Superstore Sales orders data.

In [ ]:
orders

#### Getting information (shape & data types) on the orders data.

In [ ]:
orders.info()

#### Getting column names.

In [ ]:
orders.columns

#### 1. Removal of duplicated rows.

In [ ]:
orders.duplicated().sum()

In [ ]:
orders[orders.duplicated(subset = ['Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country/Region', 'City',
       'State/Province', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit'])]

In [ ]:
orders[orders['Order ID'].isin(['US-2019-150119', 'CA-2019-153623'])]

In [ ]:
orders = orders.drop_duplicates(subset = ['Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country/Region', 'City',
       'State/Province', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit'])

#### 2. Data formatting & standardisation.

In [ ]:
orders['Row ID'].nunique()

In [ ]:
orders['Order ID'].sort_values().drop_duplicates()

In [ ]:
orders['Order Date'].sort_values().drop_duplicates()

In [ ]:
orders['Order Date'] = pd.to_datetime(orders["Order Date"], format="%m/%d/%Y")

In [ ]:
orders['Ship Date'].sort_values().drop_duplicates()

In [ ]:
orders['Ship Date'] = pd.to_datetime(orders["Ship Date"], format="%m/%d/%Y")

In [ ]:
orders['Ship Mode'].sort_values().drop_duplicates()

In [ ]:
orders['State/Province'].sort_values().drop_duplicates()

In [ ]:
orders = orders.rename(columns = {"State/Province" : "State"})

In [ ]:
orders['Postal Code'].unique()

In [ ]:
orders['Region'].sort_values().drop_duplicates()

In [ ]:
orders['Product ID'].sort_values().drop_duplicates()

In [ ]:
orders['Category'].sort_values().drop_duplicates()

In [ ]:
def left(x):
    return x.str[0:3]

orders[['Product ID', 'Category']].apply(left)

In [ ]:
orders['Sub-Category'].sort_values().drop_duplicates()

In [ ]:
orders['Product Name'].sort_values().drop_duplicates()

In [ ]:
orders['Sales'].sort_values().drop_duplicates()

In [ ]:
orders['Quantity'].sort_values().drop_duplicates()

In [ ]:
orders['Discount'].sort_values().drop_duplicates()

#### 3. Imputation of blank or null values.

In [ ]:
orders.isna().sum()

In [ ]:
#No blank or null values.

#### 4. Removal of columns irrevelant to the analysis.

In [ ]:
orders = orders.drop('Row ID', axis=1)

#### 5. Removal of rows irrevelant to the analysis.

In [ ]:
orders = orders[orders['Country/Region'] == 'United States']

## Superstore Sales Data Analysis.

#### 1. Key Performance Indicators by Year.

In [ ]:
orders["Order Year"] = orders["Order Date"].dt.year

In [ ]:
yearly_kpis = orders.groupby("Order Year").agg({"Sales": 'sum', "Quantity": 'sum', "Profit": 'sum', "Order ID": 'nunique', "Customer ID": 'nunique'}).sort_values(by = "Order Year", ascending=False).round(0)

In [ ]:
yearly_kpis = yearly_kpis.rename(columns = {"Sales":"CY Sales", "Quantity":"CY Quantity", "Profit":"CY Profit", "Order ID":"CY Orders", "Customer ID":"CY Customers"})

In [ ]:
yearly_kpis["PY Sales"] = yearly_kpis["CY Sales"].shift(-1)
yearly_kpis["PY Quantity"] = yearly_kpis["CY Quantity"].shift(-1)
yearly_kpis["PY Profit"] = yearly_kpis["CY Profit"].shift(-1)
yearly_kpis["PY Orders"] = yearly_kpis["CY Orders"].shift(-1)
yearly_kpis["PY Customers"] = yearly_kpis["CY Customers"].shift(-1)

In [ ]:
yearly_kpis["YoY Sales (%)"] = ((yearly_kpis["CY Sales"] - yearly_kpis["PY Sales"])*100/yearly_kpis["PY Sales"]).round(1)
yearly_kpis["YoY Quantity (%)"] = ((yearly_kpis["CY Quantity"] - yearly_kpis["PY Quantity"])*100/yearly_kpis["PY Quantity"]).round(1)
yearly_kpis["YoY Profit (%)"] = ((yearly_kpis["CY Profit"] - yearly_kpis["PY Profit"])*100/yearly_kpis["PY Profit"]).round(1)
yearly_kpis["YoY Orders (%)"] = ((yearly_kpis["CY Orders"] - yearly_kpis["PY Orders"])*100/yearly_kpis["PY Orders"]).round(1)
yearly_kpis["YoY Customers (%)"] = ((yearly_kpis["CY Customers"] - yearly_kpis["PY Customers"])*100/yearly_kpis["PY Customers"]).round(1)

In [ ]:
yearly_kpis = yearly_kpis[["CY Sales", "PY Sales", "YoY Sales (%)", "CY Quantity", "PY Quantity", "YoY Quantity (%)", "CY Profit", "PY Profit", "YoY Profit (%)", "CY Orders", "PY Orders", "YoY Orders (%)", "CY Customers", "PY Customers", "YoY Customers (%)", ]]

In [ ]:
yearly_kpis.reset_index(inplace=True)

In [ ]:
yearly_kpis

#### 2. Key Performance Indicators by Year & Month

In [ ]:
orders["Order Month"] = orders["Order Date"].dt.month
orders["Order Month Name"] = orders["Order Date"].dt.month_name()

In [ ]:
monthly_kpis = orders.groupby(["Order Month", "Order Month Name", "Order Year"]).agg({"Sales": 'sum', "Quantity": 'sum', "Profit": 'sum', "Order ID": 'nunique', "Customer ID": 'nunique'}).sort_values(by = ["Order Month", "Order Month Name", "Order Year"], ascending=[True, True, False]).round(0)

In [ ]:
monthly_kpis = monthly_kpis.rename(columns = {"Sales":"CY Sales", "Quantity":"CY Quantity", "Profit":"CY Profit", "Order ID":"CY Orders", "Customer ID":"CY Customers"})

In [ ]:
monthly_kpis["PY Sales"] = monthly_kpis["CY Sales"].shift(-1)
monthly_kpis["PY Quantity"] = monthly_kpis["CY Quantity"].shift(-1)
monthly_kpis["PY Profit"] = monthly_kpis["CY Profit"].shift(-1)
monthly_kpis["PY Orders"] = monthly_kpis["CY Orders"].shift(-1)
monthly_kpis["PY Customers"] = monthly_kpis["CY Customers"].shift(-1)

In [ ]:
monthly_kpis["YoY Sales (%)"] = ((monthly_kpis["CY Sales"] - monthly_kpis["PY Sales"])*100/monthly_kpis["PY Sales"]).round(1)
monthly_kpis["YoY Quantity (%)"] = ((monthly_kpis["CY Quantity"] - monthly_kpis["PY Quantity"])*100/monthly_kpis["PY Quantity"]).round(1)
monthly_kpis["YoY Profit (%)"] = ((monthly_kpis["CY Profit"] - monthly_kpis["PY Profit"])*100/monthly_kpis["PY Profit"]).round(1)
monthly_kpis["YoY Orders (%)"] = ((monthly_kpis["CY Orders"] - monthly_kpis["PY Orders"])*100/monthly_kpis["PY Orders"]).round(1)
monthly_kpis["YoY Customers (%)"] = ((monthly_kpis["CY Customers"] - monthly_kpis["PY Customers"])*100/monthly_kpis["PY Customers"]).round(1)

In [ ]:
monthly_kpis = monthly_kpis[["CY Sales", "PY Sales", "YoY Sales (%)", "CY Quantity", "PY Quantity", "YoY Quantity (%)", "CY Profit", "PY Profit", "YoY Profit (%)", "CY Orders", "PY Orders", "YoY Orders (%)", "CY Customers", "PY Customers", "YoY Customers (%)", ]]

In [ ]:
monthly_kpis.reset_index(inplace=True)

In [ ]:
monthly_kpis

#### 3. Key Perfomance Indicators by Region and State

In [ ]:
kpis_region_state = orders.groupby(["Region", "State", "Order Year"]).agg({"Sales": 'sum', "Quantity": 'sum', "Profit": 'sum', "Order ID": 'nunique', "Customer ID": 'nunique'}).sort_values(by = ["Region", "State", "Order Year"], ascending=[True, True, False]).round(0)

In [ ]:
kpis_region_state = kpis_region_state.rename(columns = {"Sales":"CY Sales", "Quantity":"CY Quantity", "Profit":"CY Profit", "Order ID":"CY Orders", "Customer ID":"CY Customers"})

In [ ]:
kpis_region_state["PY Sales"] = kpis_region_state["CY Sales"].shift(-1)
kpis_region_state["PY Quantity"] = kpis_region_state["CY Quantity"].shift(-1)
kpis_region_state["PY Profit"] = kpis_region_state["CY Profit"].shift(-1)
kpis_region_state["PY Orders"] = kpis_region_state["CY Orders"].shift(-1)
kpis_region_state["PY Customers"] = kpis_region_state["CY Customers"].shift(-1)

In [ ]:
kpis_region_state["YoY Sales (%)"] = ((kpis_region_state["CY Sales"] - kpis_region_state["PY Sales"])*100/kpis_region_state["PY Sales"]).round(1)
kpis_region_state["YoY Quantity (%)"] = ((kpis_region_state["CY Quantity"] - kpis_region_state["PY Quantity"])*100/kpis_region_state["PY Quantity"]).round(1)
kpis_region_state["YoY Profit (%)"] = ((kpis_region_state["CY Profit"] - kpis_region_state["PY Profit"])*100/kpis_region_state["PY Profit"]).round(1)
kpis_region_state["YoY Orders (%)"] = ((kpis_region_state["CY Orders"] - kpis_region_state["PY Orders"])*100/kpis_region_state["PY Orders"]).round(1)
kpis_region_state["YoY Customers (%)"] = ((kpis_region_state["CY Customers"] - kpis_region_state["PY Customers"])*100/kpis_region_state["PY Customers"]).round(1)

In [ ]:
kpis_region_state = kpis_region_state[["CY Sales", "PY Sales", "YoY Sales (%)", "CY Quantity", "PY Quantity", "YoY Quantity (%)", "CY Profit", "PY Profit", "YoY Profit (%)", "CY Orders", "PY Orders", "YoY Orders (%)", "CY Customers", "PY Customers", "YoY Customers (%)", ]]

In [ ]:
kpis_region_state.reset_index(inplace=True)

In [ ]:
kpis_region_state

#### 4. Key Perfomance Indicators by Category and Sub-category

In [ ]:
kpis_category_subcategory = orders.groupby(["Category", "Sub-Category", "Order Year"]).agg({"Sales": 'sum', "Quantity": 'sum', "Profit": 'sum', "Order ID": 'nunique', "Customer ID": 'nunique'}).sort_values(by = ["Category", "Sub-Category", "Order Year"], ascending=[True, True, False]).round(0)

In [ ]:
kpis_category_subcategory = kpis_category_subcategory.rename(columns = {"Sales":"CY Sales", "Quantity":"CY Quantity", "Profit":"CY Profit", "Order ID":"CY Orders", "Customer ID":"CY Customers"})

In [ ]:
kpis_category_subcategory["PY Sales"] = kpis_category_subcategory["CY Sales"].shift(-1)
kpis_category_subcategory["PY Quantity"] = kpis_category_subcategory["CY Quantity"].shift(-1)
kpis_category_subcategory["PY Profit"] = kpis_category_subcategory["CY Profit"].shift(-1)
kpis_category_subcategory["PY Orders"] = kpis_category_subcategory["CY Orders"].shift(-1)
kpis_category_subcategory["PY Customers"] = kpis_category_subcategory["CY Customers"].shift(-1)

In [ ]:
kpis_category_subcategory["YoY Sales (%)"] = ((kpis_category_subcategory["CY Sales"] - kpis_category_subcategory["PY Sales"])*100/kpis_category_subcategory["PY Sales"]).round(1)
kpis_category_subcategory["YoY Quantity (%)"] = ((kpis_category_subcategory["CY Quantity"] - kpis_category_subcategory["PY Quantity"])*100/kpis_category_subcategory["PY Quantity"]).round(1)
kpis_category_subcategory["YoY Profit (%)"] = ((kpis_category_subcategory["CY Profit"] - kpis_category_subcategory["PY Profit"])*100/kpis_category_subcategory["PY Profit"]).round(1)
kpis_category_subcategory["YoY Orders (%)"] = ((kpis_category_subcategory["CY Orders"] - kpis_category_subcategory["PY Orders"])*100/kpis_category_subcategory["PY Orders"]).round(1)
kpis_category_subcategory["YoY Customers (%)"] = ((kpis_category_subcategory["CY Customers"] - kpis_category_subcategory["PY Customers"])*100/kpis_category_subcategory["PY Customers"]).round(1)

In [ ]:
kpis_category_subcategory = kpis_category_subcategory[["CY Sales", "PY Sales", "YoY Sales (%)", "CY Quantity", "PY Quantity", "YoY Quantity (%)", "CY Profit", "PY Profit", "YoY Profit (%)", "CY Orders", "PY Orders", "YoY Orders (%)", "CY Customers", "PY Customers", "YoY Customers (%)", ]]

In [ ]:
kpis_category_subcategory.reset_index(inplace=True)

In [ ]:
kpis_category_subcategory

#### 5. Key Perfomance Indicators by Segment

In [ ]:
kpis_segment = orders.groupby(["Segment", "Order Year"]).agg({"Sales": 'sum', "Quantity": 'sum', "Profit": 'sum', "Order ID": 'nunique', "Customer ID": 'nunique'}).sort_values(by = ["Segment", "Order Year"], ascending=[True, False]).round(0)

In [ ]:
kpis_segment = kpis_segment.rename(columns = {"Sales":"CY Sales", "Quantity":"CY Quantity", "Profit":"CY Profit", "Order ID":"CY Orders", "Customer ID":"CY Customers"})

In [ ]:
kpis_segment["PY Sales"] = kpis_segment["CY Sales"].shift(-1)
kpis_segment["PY Quantity"] = kpis_segment["CY Quantity"].shift(-1)
kpis_segment["PY Profit"] = kpis_segment["CY Profit"].shift(-1)
kpis_segment["PY Orders"] = kpis_segment["CY Orders"].shift(-1)
kpis_segment["PY Customers"] = kpis_segment["CY Customers"].shift(-1)

In [ ]:
kpis_segment["YoY Sales (%)"] = ((kpis_segment["CY Sales"] - kpis_segment["PY Sales"])*100/kpis_segment["PY Sales"]).round(1)
kpis_segment["YoY Quantity (%)"] = ((kpis_segment["CY Quantity"] - kpis_segment["PY Quantity"])*100/kpis_segment["PY Quantity"]).round(1)
kpis_segment["YoY Profit (%)"] = ((kpis_segment["CY Profit"] - kpis_segment["PY Profit"])*100/kpis_segment["PY Profit"]).round(1)
kpis_segment["YoY Orders (%)"] = ((kpis_segment["CY Orders"] - kpis_segment["PY Orders"])*100/kpis_segment["PY Orders"]).round(1)
kpis_segment["YoY Customers (%)"] = ((kpis_segment["CY Customers"] - kpis_segment["PY Customers"])*100/kpis_segment["PY Customers"]).round(1)

In [ ]:
kpis_segment = kpis_segment[["CY Sales", "PY Sales", "YoY Sales (%)", "CY Quantity", "PY Quantity", "YoY Quantity (%)", "CY Profit", "PY Profit", "YoY Profit (%)", "CY Orders", "PY Orders", "YoY Orders (%)", "CY Customers", "PY Customers", "YoY Customers (%)", ]]

In [ ]:
kpis_segment.reset_index(inplace=True)

In [ ]:
kpis_segment

#### 6. Key Perfomance Indicators by Ship Mode

In [ ]:
kpis_shipmode = orders.groupby(["Ship Mode", "Order Year"]).agg({"Sales": 'sum', "Quantity": 'sum', "Profit": 'sum', "Order ID": 'nunique', "Customer ID": 'nunique'}).sort_values(by = ["Ship Mode", "Order Year"], ascending=[True, False]).round(0)

In [ ]:
kpis_shipmode = kpis_shipmode.rename(columns = {"Sales":"CY Sales", "Quantity":"CY Quantity", "Profit":"CY Profit", "Order ID":"CY Orders", "Customer ID":"CY Customers"})

In [ ]:
kpis_shipmode["PY Sales"] = kpis_shipmode["CY Sales"].shift(-1)
kpis_shipmode["PY Quantity"] = kpis_shipmode["CY Quantity"].shift(-1)
kpis_shipmode["PY Profit"] = kpis_shipmode["CY Profit"].shift(-1)
kpis_shipmode["PY Orders"] = kpis_shipmode["CY Orders"].shift(-1)
kpis_shipmode["PY Customers"] = kpis_shipmode["CY Customers"].shift(-1)

In [ ]:
kpis_shipmode["YoY Sales (%)"] = ((kpis_shipmode["CY Sales"] - kpis_shipmode["PY Sales"])*100/kpis_shipmode["PY Sales"]).round(1)
kpis_shipmode["YoY Quantity (%)"] = ((kpis_shipmode["CY Quantity"] - kpis_shipmode["PY Quantity"])*100/kpis_shipmode["PY Quantity"]).round(1)
kpis_shipmode["YoY Profit (%)"] = ((kpis_shipmode["CY Profit"] - kpis_shipmode["PY Profit"])*100/kpis_shipmode["PY Profit"]).round(1)
kpis_shipmode["YoY Orders (%)"] = ((kpis_shipmode["CY Orders"] - kpis_shipmode["PY Orders"])*100/kpis_shipmode["PY Orders"]).round(1)
kpis_shipmode["YoY Customers (%)"] = ((kpis_shipmode["CY Customers"] - kpis_shipmode["PY Customers"])*100/kpis_shipmode["PY Customers"]).round(1)

In [ ]:
kpis_shipmode = kpis_shipmode[["CY Sales", "PY Sales", "YoY Sales (%)", "CY Quantity", "PY Quantity", "YoY Quantity (%)", "CY Profit", "PY Profit", "YoY Profit (%)", "CY Orders", "PY Orders", "YoY Orders (%)", "CY Customers", "PY Customers", "YoY Customers (%)", ]]

In [ ]:
kpis_shipmode.reset_index(inplace=True)

In [ ]:
kpis_shipmode